In [91]:
import mysql.connector

In [92]:
cnx = mysql.connector.connect(user='root', password='secret',
                              host='localhost',
                              database='sakila')

In [93]:
import pandas as pd

In [94]:
schema = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME like '%' and TABLE_SCHEMA = 'sakila'"

In [95]:
df = pd.read_sql(sql=schema, con=cnx, index_col=None, coerce_float=True, params=None, parse_dates=None, 
            columns=None, chunksize=None)

In [96]:
df

,TABLE_NAME
0,actor
1,actor_info
2,address
3,category
4,city
5,country
6,customer
7,customer_list
8,film
9,film_actor


In [109]:
table1 = df['TABLE_NAME'][7]
table2 = df['TABLE_NAME'][8]

In [110]:
query_identify1 = "desc %s"%table1
query_identify2 = "desc %s"%table2


In [111]:
meta1 = pd.read_sql(sql=query_identify1, con=cnx, index_col=None, coerce_float=True, params=None, parse_dates=None, 
            columns=None, chunksize=None)['Type']
meta2 = pd.read_sql(sql=query_identify2, con=cnx, index_col=None, coerce_float=True, params=None, parse_dates=None, 
            columns=None, chunksize=None)['Type']

In [112]:
print(meta1)

0    smallint(5) unsigned
1             varchar(91)
2             varchar(50)
3             varchar(10)
4             varchar(20)
5             varchar(50)
6             varchar(50)
7              varchar(6)
8     tinyint(3) unsigned
Name: Type, dtype: object


In [113]:
print(meta2)

0                                  smallint(5) unsigned
1                                          varchar(255)
2                                                  text
3                                               year(4)
4                                   tinyint(3) unsigned
5                                   tinyint(3) unsigned
6                                   tinyint(3) unsigned
7                                          decimal(4,2)
8                                  smallint(5) unsigned
9                                          decimal(5,2)
10                   enum('G','PG','PG-13','R','NC-17')
11    set('Trailers','Commentaries','Deleted Scenes'...
12                                            timestamp
Name: Type, dtype: object


In [114]:
dict = {}

In [115]:
# for i,c1 in enumerate(meta1):
#     if(str(c1) != 'varchar(45)'):
#         for j,c2 in enumerate(meta2):
#             if(str(c2) != 'varchar(45)'):
#                 if(str(c1) == str(c2)):
#                    print("Column %d : Table 1 <=> Column %d : Table 2"%(i+1,j+1))

In [117]:
for i,c1 in enumerate(meta1):
        dict[i] = []
        for j,c2 in enumerate(meta2):
                if(str(c1) == str(c2)):
                    dict[i].append(j) 
                    print("Column %d : Table 1 <=> Column %d : Table 2"%(i,j))

Column 0 : Table 1 <=> Column 0 : Table 2
Column 0 : Table 1 <=> Column 8 : Table 2
Column 8 : Table 1 <=> Column 4 : Table 2
Column 8 : Table 1 <=> Column 5 : Table 2
Column 8 : Table 1 <=> Column 6 : Table 2


In [118]:
table_data_query = "select * from (%s)"%table1

bro = pd.read_sql(sql=table_data_query, con=cnx, index_col=None, coerce_float=True, params=None, parse_dates=None, 
            columns=None, chunksize=None)

In [119]:
dict

{0: [0, 8], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [4, 5, 6]}

In [120]:
percent = 0.05

In [123]:
for k in dict.keys():
    l = dict[k]
    col1 = bro.columns[k]
    df_c1 = pd.DataFrame(bro[col1].sample(frac=percent, replace=False))
    for v in l:
        col2 = bro.columns[v]
        df_c2 = pd.DataFrame(bro[col2].sample(frac=percent, replace=False))
        print(levenshtein(df_c1, df_c2))

0
1
1
1
1


In [122]:
# def levenshtein(s1, s2):
#     if len(s1) < len(s2):
#         return levenshtein(s2, s1)

#     # len(s1) >= len(s2)
#     if len(s2) == 0:
#         return len(s1)

#     previous_row = range(len(s2) + 1)
#     for i, c1 in enumerate(s1):
#         current_row = [i + 1]
#         for j, c2 in enumerate(s2):
#             insertions = previous_row[j + 1] + 1
#             deletions = current_row[j] + 1
#             substitutions = previous_row[j] + (c1 != c2)
#             current_row.append(min(insertions, deletions, substitutions))
#         previous_row = current_row
    
#     return previous_row[-1]